In [1]:
###############################
# Hoshen Kopelman Algorithm
###############################
function unionHK(i, j, labels)
    labels[findHK(maximum([i,j]), labels)] = findHK(minimum([i,j]), labels)
    
#     max = maximum([findHK(i, labels), findHK(j, labels)])
#     min = minimum([findHK(i, labels), findHK(j, labels)])
#     labels[max] = min
end

function findHK(x, labels)
    y = x
    while (labels[y] != y)
        y = labels[y]
    end

    while (labels[x] != x) 
        z = labels[x]
        labels[x] = y
        x = z
    end
    
    return y
end

function HK(occupied)
    n_rows, n_columns = size(occupied)
    labels = collect(1:n_rows * n_columns)
    largest_label = 0
    for i in 1:n_rows
        for j in 1:n_columns
            if occupied[i,j] == 1
                if j == 1
                    left = 0
                else
                    left = occupied[i,j-1]
                end
                
                if i == 1
                    above = 0
                else
                    above = occupied[i-1,j]
                end
                
                if left == 0 && above == 0
                    largest_label += 1
                    occupied[i,j] = largest_label
                elseif left != 0 && above == 0
                    occupied[i,j] = findHK(left, labels)
                elseif left == 0 && above != 0
                    occupied[i,j] = findHK(above, labels)
                else
                    unionHK(left, above, labels)
                    occupied[i,j] = findHK(left, labels)
                end
            end
        end
    end
    return largest_label, labels
end

function HK_simd(occupied)
    n_rows, n_columns = size(occupied)
    labels = collect(1:n_rows * n_columns)
    largest_label = 0
    @simd for i in 1:n_rows
        @simd for j in 1:n_columns
            if occupied[i,j] == 1
                if j == 1
                    left = 0
                else
                    @inbounds left = occupied[i,j-1]
                end
                
                if i == 1
                    above = 0
                else
                    @inbounds above = occupied[i-1,j]
                end
                
                if left == 0 && above == 0
                    largest_label += 1
                    @inbounds occupied[i,j] = largest_label
                elseif left != 0 && above == 0
                    @inbounds occupied[i,j] = findHK(left, labels)
                elseif left == 0 && above != 0
                    @inbounds occupied[i,j] = findHK(above, labels)
                else
                    unionHK(left, above, labels)
                    @inbounds occupied[i,j] = findHK(left, labels)
                end
            end
        end
    end
    return largest_label, labels
end

#########################
# recursive function
#########################
function recursive(occupied)
    (row, column) = size(occupied)
    labelnum = 101
    for i in 1:row, j in 1:column
        if occupied[i,j] == 1
            checkcluster(i, j, labelnum, occupied)
            labelnum += 1
        end
    end
    
    return labelnum - 100
end

# square lattice nearest neighbor
function checkcluster(i::Int, j::Int, labelnum::Int, occupied::Matrix{Int})
    (row,  column) = size(occupied)
    if occupied[i,j] == 1
        occupied[i,j] = labelnum
        if j < column; checkcluster(i, j+1, labelnum, occupied); end
        if 1 < j; checkcluster(i, j-1, labelnum, occupied); end
        if i < row; checkcluster(i+1, j, labelnum, occupied); end
        if 1 < i; checkcluster(i-1, j, labelnum, occupied); end
    end
end


# instead of recursive
# pick up search list
function pickup(occupied::Matrix{Int})
    (row, column) = size(occupied)
    labelnum = 101
    for i in 1:row, j in 1:column
        if occupied[i,j] == 1
            searchlist = checkcluster_pickup(occupied, i,j, labelnum)
            occupied[i,j] = labelnum

            while searchlist != []
                tmppos = pop!(searchlist)
                searchlist =  [ searchlist; checkcluster_pickup(occupied, tmppos[1], tmppos[2], labelnum) ]
                occupied[tmppos...] = labelnum
            end

            labelnum += 1
        end
    end
end

function checkcluster_pickup(occupied::Matrix{Int}, i::Int, j::Int, labelnum::Int)
    (row, column) = size(occupied)
    searchlist = Array{Array{Int64, 1}, 1}()

    if occupied[i,j] == 1
        if j < column && occupied[i, j+1] == 1; push!(searchlist, [i, j+1]); end
        if 1 < j && occupied[i, j-1] == 1; push!(searchlist, [i, j-1]); end
        if i < row && occupied[i+1, j] == 1; push!(searchlist, [i+1, j]); end
        if 1 < i && occupied[i-1, j] == 1; push!(searchlist, [i-1, j]); end
    end

    return searchlist
end

checkcluster_pickup (generic function with 1 method)

In [8]:
occupied = [
 0  1  1  0  0  0  1
 1  0  0  0  0  1  0
 0  1  1  0  1  0  1
 0  1  1  1  0  0  0
 1  0  1  0  0  1  1
 0  1  0  0  1  1  1
 1  1  1  1  1  1  0]
HKA = occupied[:,:]; HKA_simd = occupied[:,:]; rec = occupied[:,:]; pick = occupied[:,:]
@time max, label = HK(HKA)
modHKA = HKA[:,:]
@time max_simd, label_simd = HK_simd(HKA_simd)

@time recursive(rec)
@time pickup(pick)

@time for i in 1:length(label)
    x = i
   while (label[x] != x)
       x = label[x]
   end
   
   label[i] = x
end

for i in 1:length(modHKA)
   if modHKA[i] != 0
       modHKA[i] = label[modHKA[i]]
   end
end
rec .-= 100
rec[rec .== -100] .= 0;
pick .-= 100
pick[pick .== -100] .= 0;

  0.000018 seconds (21 allocations: 1.938 KB)
  0.000011 seconds (21 allocations: 1.938 KB)
  0.000003 seconds (4 allocations: 160 bytes)
  0.000015 seconds (134 allocations: 9.781 KB)
  0.000030 seconds (50 allocations: 1.563 KB)


In [3]:
occupied

7×7 Array{Int64,2}:
 0  1  1  0  0  0  1
 1  0  0  0  0  1  0
 0  1  1  0  1  0  1
 0  1  1  1  0  0  0
 1  0  1  0  0  1  1
 0  1  0  0  1  1  1
 1  1  1  1  1  1  0

In [11]:
rec

7×7 Array{Int64,2}:
 0  1  1  0  0  0  2
 3  0  0  0  0  4  0
 0  5  5  0  6  0  7
 0  5  5  5  0  0  0
 8  0  5  0  0  9  9
 0  9  0  0  9  9  9
 9  9  9  9  9  9  0

In [12]:
pick

7×7 Array{Int64,2}:
 0  1  1  0  0  0  2
 3  0  0  0  0  4  0
 0  5  5  0  6  0  7
 0  5  5  5  0  0  0
 8  0  5  0  0  9  9
 0  9  0  0  9  9  9
 9  9  9  9  9  9  0

In [13]:
HKA

7×7 Array{Int64,2}:
  0   1   1   0   0   0  2
  3   0   0   0   0   4  0
  0   5   5   0   6   0  7
  0   5   5   5   0   0  0
  8   0   5   0   0   9  9
  0  10   0   0  11   9  9
 12  10  10  10  10  10  0

In [14]:
modHKA

7×7 Array{Int64,2}:
  0   1   1   0   0   0   2
  3   0   0   0   0   4   0
  0   5   5   0   6   0   7
  0   5   5   5   0   0   0
  8   0   5   0   0  10  10
  0  10   0   0  10  10  10
 10  10  10  10  10  10   0

In [15]:
modHKA == rec

false

In [16]:
println(label)

[1,2,3,4,5,6,7,8,10,10,10,10,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49]


# occupiedがrandom

In [ ]:
N = 10000
occupied = (rand(N, N) .< 0.5) * 1
HKA = occupied[:,:]; rec = occupied[:,:]
@time max, label = HK(HKA)
modHKA = HKA[:,:]
@time recursive(rec)
@time for i in 0:length(label)-1
   x = length(label)-i
   while (label[x] != x)
       x = label[x]
   end
   
   label[length(label)-i] = x
end

@time for i in 1:length(modHKA)
   if modHKA[i] != 0
       modHKA[i] = label[modHKA[i]]
   end
end
rec .-= 100
rec[rec .== -100] .= 0;

In [ ]:
occupied

In [ ]:
HKA

In [ ]:
modHKA

In [ ]:
HKA == modHKA

In [ ]:
rec

In [ ]:
modHKA == rec

# conventional algorithm

In [60]:
N = 7; srand(2); a = (rand(N,N) .< 0.5) * 1

7×7 Array{Int64,2}:
 1  1  0  1  0  1  1
 0  0  1  0  1  1  1
 1  0  0  1  1  0  0
 0  1  1  1  0  1  1
 0  0  0  0  0  1  0
 0  1  1  1  1  1  0
 0  0  1  1  1  0  0

In [61]:
function convscanfw(i,j,a,curlab)
    _N = size(a)[1]
    if 1 < j; left = a[i,j-1];else; left = 0; end
    if 1 < i; above = a[i-1,j]; else; above = 0; end
    
    min = minimum([left, above])
    if left != 0 && above != 0
        a[i,j] = min
    elseif left!= 0 && above == 0
            a[i,j] = left
    elseif above != 0 && left == 0
        a[i,j] = above
    else
        curlab += 1
        a[i,j] = curlab
    end
    
    return curlab
end

convscanfw (generic function with 1 method)

In [62]:
curlab = 0
for i in 1:N, j in 1:N
    if a[i,j] != 0
        curlab = convscanfw(i,j,a,curlab)
    end
end

In [63]:
a

7×7 Array{Int64,2}:
 1   1   0   2   0  3  3
 0   0   4   0   5  3  3
 6   0   0   7   5  0  0
 0   8   8   7   0  9  9
 0   0   0   0   0  9  0
 0  10  10  10  10  9  0
 0   0  10  10  10  0  0

In [64]:
function convscanbw(i,j,a)
    _N = size(a)[1]
    if j < _N; right = a[i,j+1]; else; right = 0; end
    if i < _N; below = a[i+1,j]; else; below = 0; end
    
    min = minimum([below, right])
    if right != 0 && below != 0
        a[i,j] = min
    elseif right != 0 && below == 0
        a[i,j] = right
    elseif below != 0 && right == 0
        a[i,j] = below
    else
        
    end

end

convscanbw (generic function with 2 methods)

In [65]:
for i in 0:N-1, j in 0:N-1
    if a[N-i,N-j] != 0
        curlab = convscanbw(N-i,N-j,a)
    end
end

In [66]:
a

7×7 Array{Int64,2}:
 1  1   0   2   0  3  3
 0  0   4   0   3  3  3
 6  0   0   5   5  0  0
 0  7   7   7   0  9  9
 0  0   0   0   0  9  0
 0  9   9   9   9  9  0
 0  0  10  10  10  0  0

In [67]:
function convscanfw2(i,j,a)
    _N = size(a)[1]
    if 1 < j; left = a[i,j-1];else; left = 0; end
    if 1 < i; above = a[i-1,j]; else; above = 0; end
    
    min = minimum([left, above])
    if left != 0 && above != 0
        a[i,j] = min
    elseif left!= 0 && above == 0
            a[i,j] = left
    elseif above != 0 && left == 0
        a[i,j] = above
    else
        
    end
end

convscanfw2 (generic function with 1 method)

In [68]:
for i in 1:N, j in 1:N
    if a[i,j] != 0
        convscanfw2(i,j,a)
    end
end

In [69]:
a

7×7 Array{Int64,2}:
 1  1  0  2  0  3  3
 0  0  4  0  3  3  3
 6  0  0  5  3  0  0
 0  7  7  5  0  9  9
 0  0  0  0  0  9  0
 0  9  9  9  9  9  0
 0  0  9  9  9  0  0

In [70]:
for i in 0:N-1, j in 0:N-1
    if a[N-i,N-j] != 0
        curlab = convscanbw(N-i,N-j,a)
    end
end

In [71]:
a

7×7 Array{Int64,2}:
 1  1  0  2  0  3  3
 0  0  4  0  3  3  3
 6  0  0  3  3  0  0
 0  5  5  5  0  9  9
 0  0  0  0  0  9  0
 0  9  9  9  9  9  0
 0  0  9  9  9  0  0

In [72]:
for i in 1:N, j in 1:N
    if a[i,j] != 0
        convscanfw2(i,j,a)
    end
end

In [73]:
a

7×7 Array{Int64,2}:
 1  1  0  2  0  3  3
 0  0  4  0  3  3  3
 6  0  0  3  3  0  0
 0  5  5  3  0  9  9
 0  0  0  0  0  9  0
 0  9  9  9  9  9  0
 0  0  9  9  9  0  0

In [74]:
for i in 0:N-1, j in 0:N-1
    if a[N-i,N-j] != 0
        curlab = convscanbw(N-i,N-j,a)
    end
end

In [75]:
a

7×7 Array{Int64,2}:
 1  1  0  2  0  3  3
 0  0  4  0  3  3  3
 6  0  0  3  3  0  0
 0  3  3  3  0  9  9
 0  0  0  0  0  9  0
 0  9  9  9  9  9  0
 0  0  9  9  9  0  0